In [0]:
spark.sql("use finance_catalog.bronze")

In [0]:
# STREAMING Bronze Layer - Auto Loader
df_bronze = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaLocation", "abfss://catalog@adlsdevfinancesensitive.dfs.core.windows.net/schema/")
    .load("abfss://raw@adlsdevfinancesensitive.dfs.core.windows.net/input/")
    
)


In [0]:

df_bronze.writeStream.format("delta") \
    .option("checkpointLocation", "abfss://catalog@adlsdevfinancesensitive.dfs.core.windows.net/checkpoints/") \
    .outputMode("Append") \
    .table("warehouse_retail_bronze")

In [0]:
%sql
select * from warehouse_retail_bronze 

In [0]:
silver_table_name = "finance_catalog.silver.silver_orders_processed"

# Check if table exists
from pyspark.sql.functions import col, to_timestamp, lit, upper, current_timestamp
from pyspark.sql.types import StringType, IntegerType, DoubleType, TimestampType



if not spark.catalog.tableExists(silver_table_name):
  spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {silver_table_name} (
        transaction_id STRING,
        customer_id STRING,
        product_id STRING,
        amount DOUBLE,
        transaction_date TIMESTAMP,
        region STRING
    ) USING DELTA
    """)
   
  print(f"Created silver table: {silver_table_name}")
else:
  print(f"Silver table already exists: {silver_table_name}")


   


In [0]:
%sql
MERGE INTO finance_catalog.silver.silver_orders_processed AS target
USING warehouse_retail_bronze AS source
ON target.transaction_id = source.transaction_id
WHEN MATCHED THEN UPDATE SET
    target.transaction_id = source.transaction_id,
    target.customer_id = source.customer_id,
    target.product_id = source.product_id,
    target.amount = source.amount,
    target.transaction_date = source.transaction_date,
    target.region = source.region
WHEN NOT MATCHED THEN INSERT *



In [0]:
%sql
select transaction_id,count(*) from finance_catalog.silver.silver_orders_processed group by transaction_id having count(*) 

In [0]:
%sql
select * from finance_catalog.silver.silver_orders_processed 